In [1]:
!pip install scikit-surprise

# 컨텐츠 기반 필터링

In [9]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
#os파일 데이터를 surprise데이터 세트로 로딩
import pandas as pd

ratings = pd.read_csv('../DataSet/ratings_small/ratings_small.csv')
#인덱스와 헤더를 제거해서 새로운 파일 생성
ratings.to_csv('../DataSet/ratings_small/sup_ratings_small.csv', index=False, header=False)

In [6]:
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [8]:
#reader클래스에 어떤 데이터인지 알려줌
from surprise import Reader
from surprise import Dataset

#line_format은 어떤 칼럼들이 있는지, sep은 무엇으로 구분되어있는지, 평점의 단위는 최소 0.5 최대 5
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5.0))
#reader를 통해 파싱하면서 로딩(3개의 칼럼만 로딩되므로 timestamp는 자동으로 제외)
data=Dataset.load_from_file('../DataSet/ratings_small/sup_ratings_small.csv', reader=reader)

In [10]:
#SVD행렬 분해 기법을 통해서 추천 예측, 잠재 요인 크기 K값을 나타내는 n_factors는 50으로 설정

#수행 시마다 동일하게 데이터를 분할하기 위해 random_state값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

#수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

#학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

In [11]:
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction 결과의 최초 5개 추출


[Prediction(uid='30', iid='254', r_ui=3.0, est=3.7092901755922996, details={'was_impossible': False}),
 Prediction(uid='652', iid='2626', r_ui=5.0, est=4.172573234363497, details={'was_impossible': False}),
 Prediction(uid='466', iid='2174', r_ui=3.0, est=3.673029601193435, details={'was_impossible': False}),
 Prediction(uid='561', iid='3438', r_ui=4.5, est=3.1203161578977214, details={'was_impossible': False}),
 Prediction(uid='529', iid='34552', r_ui=4.0, est=3.407430180522673, details={'was_impossible': False})]

Surprise패키지의 predict()메서드를 통해서 추천 예측을 해보겠습니다. predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측하는데 개별 사용자 아이디, 아이템 아이디를 입력하면 추천 예측 평점을 포함한 정보를 반환합니다.
주의할 점은 기존 평점 정보(r_ui)는 선택 사항이며 사용자 아이디, 아이템 아이디는 문자열로 입력해야 합니다.

In [12]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.81   {'was_impossible': False}


surprise에서 각각의 알고리즘에 따른 RMSE,MAE,TIME에 대한 정보들이 있습니다.( http://surpriselib.com/ )
위에서 사용한 SVD를 한번 살펴보면 SVD의 비용함수는 사용자 베이스라인 편향성을 감안한 평점 예측에 Regularization을 적용한 것입니다.
지원하는 알고리즘들 보면 SVD++알고리즘의 RMSE,MAE성적이 가장 좋지만 시간이 상대적으로 오래걸려 데이터가 많이 클 경우 사용하기 어려운 알고리즘으로 보입니다. SVD++를 제외하면 SVD와 k-NN Baselind이 가장 선능평가 수치가 좋습니다.
Baseline이라는 의미는 각 개인이 평점을 부여하는 성향을 반영해 평점을 계산하는 방식을 말합니다.

## Baseline평점
베이스라인 평점은 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것입니다.
보통 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산됩니다.

전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점

## 교차 검증과 하이퍼 파라미터 튜닝
교차검증과 하이퍼 파라미터 튜닝을 위해 cross_validate()와 GridSearchCV클래스를 제공합니다. 먼저 cross_validate()는 인자로 알고리즘 객체, 데이터, 성능 평가 방법, 폴드 데이터 세트 개수(cv)를 입력합니다.

In [13]:
from surprise.model_selection import cross_validate

#위에서 사용했던 data와 algo사용, MAE는 오차들의 절댓값의 평균을 구한다는 점에서 모든 examples에 대한 오차에 동일한 가중치를 부여,
#RMSE는 각 example에 제곱을 취한 뒤 평균을 구하고 그것에 루트를 씌우면서 각 오차가 다른 가중치를 갖게 된다.
#verbose는 학습진행상황을 보여주는 것에 대한 여부이다.
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8921  0.8933  0.9007  0.8894  0.8883  0.8928  0.0044  
MAE (testset)     0.6860  0.6876  0.6912  0.6860  0.6849  0.6871  0.0022  
Fit time          0.84    0.82    0.81    0.79    0.79    0.81    0.02    
Test time         0.30    0.12    0.12    0.27    0.13    0.19    0.08    


{'test_rmse': array([0.89210069, 0.89327247, 0.90068642, 0.88935189, 0.88833858]),
 'test_mae': array([0.68601049, 0.68758086, 0.6911832 , 0.68596558, 0.68487759]),
 'fit_time': (0.8422880172729492,
  0.8204176425933838,
  0.805535078048706,
  0.7914371490478516,
  0.790593147277832),
 'test_time': (0.3001992702484131,
  0.12467336654663086,
  0.12380146980285645,
  0.26628708839416504,
  0.12570428848266602)}

In [15]:
#GridSearchCv클래스는 교차 검증을 통한 하이퍼 파라미터 최적화 수행, SVD의 경우 반복횟수를 지정하는 n_epochs와
#잠재요인 크기를 지정하는 n_factors를 튜닝
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs' : [20,40,60], 'n_factors' : [50,100,200]}

gs = GridSearchCV(SVD, param_grid, measures=['RMSE', 'MAE'], cv=3)
gs.fit(data)

print(gs.best_score['rmse']) #최고 RMSE Evaluation점수
print(gs.best_params['rmse']) #그때의 하이퍼 파라미터

0.8979861252486859
{'n_epochs': 20, 'n_factors': 50}


이번엔 데이터를 학습데이터와 테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용하겠습니다. 이를 위해 DatasetAutoFolds클래스를 이용합니다.

In [19]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5,5))

data_folds = DatasetAutoFolds(ratings_file='../DataSet/ratings_small/sup_ratings_small.csv', reader=reader)

#전체 데이터를 학습 데이터로 생성
trainset = data_folds.build_full_trainset()

In [20]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

특정 사용자에 영화를 추천하기 위해 아직 보지 않은 영화 목록을 확인해보겠습니다. 특정사용자는 userId=15 입니다. 15번유저가 아직 평점을 매기지 않은 영화를 movieId=8844로 선정한 뒤 예측 평점을 계산해보겠습니다.

In [40]:
#영화 데이터 로딩
movies = pd.read_csv('../DataSet/movies_metadata/movies_metadata.csv', low_memory=False)

print(movies.loc[movies['id']=='8844'])

   adult belongs_to_collection    budget  \
1  False                   NaN  65000000   

                                              genres homepage    id  \
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...      NaN  8844   

     imdb_id original_language original_title  \
1  tt0113497                en        Jumanji   

                                            overview  ... release_date  \
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   

       revenue runtime                                   spoken_languages  \
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   

     status                                    tagline    title  video  \
1  Released  Roll the dice and unleash the excitement!  Jumanji  False   

  vote_average vote_count  
1          6.9     2413.0  

[1 rows x 24 columns]


In [44]:
#15번 유저의 movieId데이터를 추출해서 movieId=8844데이터가 있는지 확인
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==8844].count()==0 :
    print('사용자 아이디 15는 영화 아이디 8844의 평점 없음')
    
print(movies[movies['id']=='8844'])

사용자 아이디 15는 영화 아이디 8844의 평점 없음
   adult belongs_to_collection    budget  \
1  False                   NaN  65000000   

                                              genres homepage    id  \
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...      NaN  8844   

     imdb_id original_language original_title  \
1  tt0113497                en        Jumanji   

                                            overview  ... release_date  \
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   

       revenue runtime                                   spoken_languages  \
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   

     status                                    tagline    title  video  \
1  Released  Roll the dice and unleash the excitement!  Jumanji  False   

  vote_average vote_count  
1          6.9     2413.0  

[1 rows x 24 columns]


In [42]:
#predict()메서드를 이용해서 추천 예상평점 알아보기
uid=str(15)
iid=str(8844)

pred=algo.predict(uid,iid,verbose=True)

user: 15         item: 8844       r_ui = None   est = 2.34   {'was_impossible': False}


추천 예측 평점은 2.34입니다. 사용자가 평점을 매기지 않은 영화의 추천 예측 평점을 간단하게 구하는 방법을 알았으니 이제 사용자가 평점을 매기지 않은 전체 영화를 추출한 뒤에 예측 평점 순으로 영화를 추천해보겠습니다. 

In [46]:
def get_unseen_surprise(ratings,movies,userId) :
    #입력 값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    
    #모든 영화의 movieId를 리스트로 생성
    total_movies = movies['id'].tolist()
    
    #모든 영화의 movieId중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies=[movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수 : ', len(seen_movies), '추천 대상 영화 수 : ', len(unseen_movies), '전체 영화 수 : ', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings,movies,15)

평점 매긴 영화 수 :  1700 추천 대상 영화 수 :  45466 전체 영화 수 :  45466


recomm함수 테스트중

In [61]:
predictions = [algo.predict(str(ratings['userId']), str(ratings['movieId'])) for movieId in unseen_movies]

In [69]:
predictions[:10]

[Prediction(uid='0           1\n1           1\n2           1\n3           1\n4           1\n         ... \n99999     671\n100000    671\n100001    671\n100002    671\n100003    671\nName: userId, Length: 100004, dtype: int64', iid='0           31\n1         1029\n2         1061\n3         1129\n4         1172\n          ... \n99999     6268\n100000    6269\n100001    6365\n100002    6385\n100003    6565\nName: movieId, Length: 100004, dtype: int64', r_ui=None, est=3.543608255669773, details={'was_impossible': False}),
 Prediction(uid='0           1\n1           1\n2           1\n3           1\n4           1\n         ... \n99999     671\n100000    671\n100001    671\n100002    671\n100003    671\nName: userId, Length: 100004, dtype: int64', iid='0           31\n1         1029\n2         1061\n3         1129\n4         1172\n          ... \n99999     6268\n100000    6269\n100001    6365\n100002    6385\n100003    6565\nName: movieId, Length: 100004, dtype: int64', r_ui=None, est=3.54360

In [65]:
def sortkey_est(pred):
    return pred.est

predictions.sort(key=sortkey_est, reverse=True) #정렬도 안되는데
top_predictions=predictions[:10]

In [66]:
top_predictions

[Prediction(uid='0           1\n1           1\n2           1\n3           1\n4           1\n         ... \n99999     671\n100000    671\n100001    671\n100002    671\n100003    671\nName: userId, Length: 100004, dtype: int64', iid='0           31\n1         1029\n2         1061\n3         1129\n4         1172\n          ... \n99999     6268\n100000    6269\n100001    6365\n100002    6385\n100003    6565\nName: movieId, Length: 100004, dtype: int64', r_ui=None, est=3.543608255669773, details={'was_impossible': False}),
 Prediction(uid='0           1\n1           1\n2           1\n3           1\n4           1\n         ... \n99999     671\n100000    671\n100001    671\n100002    671\n100003    671\nName: userId, Length: 100004, dtype: int64', iid='0           31\n1         1029\n2         1061\n3         1129\n4         1172\n          ... \n99999     6268\n100000    6269\n100001    6365\n100002    6385\n100003    6565\nName: movieId, Length: 100004, dtype: int64', r_ui=None, est=3.54360

In [88]:
x = pred.iid
x = float(x)
x = int(x)
top_movie_ids = [int(x) for pred in top_predictions]
top_movie_rating = [pred.est for pred in top_predictions]
top_movie_titles = movies[movies.id.isin(top_movie_ids)]['title']

In [74]:
top_movie_preds = [ (id,title,rating) for id,title,rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

In [85]:
top_movie_titles #아마 movies에서는 id값들이 전부 str형이라서 그런듯 근데 int형 정수를 넣어주니까 안돌아가는듯

Series([], Name: title, dtype: object)

In [87]:
top_movie_rating

[3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773,
 3.543608255669773]

In [89]:
top_movie_ids #값 복사까지??

[8844, 8844, 8844, 8844, 8844, 8844, 8844, 8844, 8844, 8844]

recomm함수 테스트 종료 결과값 확인

In [55]:
#SVD를 이용해 높은 예측 평점을 가진 순으로 영화를 추천하는 함수
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    
    #알고리즘 객체의 predict()메서드를 평점이 없는 영화에 반복 수행한 결과를 list객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    #predictions list 객체는 surprise의 Predictions객체를 원소로 갖고 있음
    # [Prediction(uid='15', iid='1', est=2.34), Prediction(uid='15', iid='2', est)....]
    
    #이를 est값으로 정렬하기 위해서 아래의 sortkey_est함수를 정의함
    #sortkey_est함수는 list객체의 sort()함수의 키 값으로 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est
    
    #sortkey_est() 반환 값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top_n]
    
    #top_n으로 추출된 영화의 정보추출. 영화 아이디, 추천예상평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.id.isin(top_movie_ids)]['original_title']
    
    top_movie_preds = [ (id,title,rating) for id,title,rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [56]:
unseen_movies = get_unseen_surprise(ratings, movies, 15)

평점 매긴 영화 수 :  1700 추천 대상 영화 수 :  45466 전체 영화 수 :  45466


In [57]:
top_movie_preds = recomm_movie_by_surprise(algo,15,unseen_movies,top_n=10)
print(top_movie_preds)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

[]
#### Top-10 추천 영화 리스트 ####
